In [2]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV

# --- 1. Cargar imagen SAR (VV y VH) ---
ruta_imagen = "'/content/drive/My Drive/Colab Notebooks/sentinel1_vv_vh.tif"  # Debe tener 2 bandas: VV, VH
with rasterio.open(ruta_imagen) as src:
    sar_img = src.read()  # (bandas, filas, columnas)
    profile = src.profile
sar_img = np.transpose(sar_img, (1, 2, 0))  # (H, W, B)

# --- 2. Crear etiquetas artificiales (suelo = 0, vegetación = 1) ---
labels = np.zeros((sar_img.shape[0], sar_img.shape[1]), dtype=np.uint8)
labels[30:70, 30:70] = 1  # Vegetación en el centro

# --- 3. Preprocesamiento ---
X = sar_img.reshape(-1, 2)
y = labels.flatten()

# Eliminar valores 0 o NaN (opcional para datos reales)
valid = np.all(X > 0, axis=1)
X = X[valid]
y = y[valid]

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- 4. Separar conjunto de entrenamiento y prueba ---
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, stratify=y, random_state=42)

# --- 5. Ajuste de SVM con búsqueda de hiperparámetros ---
params = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 0.1, 1],
    'kernel': ['rbf']
}
svc = SVC(class_weight='balanced')
grid = GridSearchCV(svc, params, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

print("Mejores parámetros:", grid.best_params_)

# --- 6. Evaluación ---
y_pred = grid.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# --- 7. Clasificación de toda la imagen ---
X_all_scaled = scaler.transform(X)
prediction_all = grid.predict(X_all_scaled)

# --- 8. Reconstruir imagen de clasificación ---
classified = np.full((sar_img.shape[0] * sar_img.shape[1]), 255, dtype=np.uint8)
classified[valid] = prediction_all
classified = classified.reshape(sar_img.shape[:2])

# --- 9. Guardar y visualizar resultado ---
output = "clasificacion_svm.tif"
profile.update(dtype='uint8', count=1)
with rasterio.open(output, 'w', **profile) as dst:
    dst.write(classified, 1)

plt.figure(figsize=(8, 6))
plt.imshow(classified, cmap='Spectral')
plt.title("Clasificación SVM (VV + VH)")
plt.axis('off')
plt.colorbar()
plt.show()


RasterioIOError: '/content/drive/My Drive/Colab Notebooks/sentinel1_vv_vh.tif: No such file or directory